In [ ]:
import pandas as pd


In [ ]:


basic = pd.read_csv('name.basics.tsv', sep='\t')


ratings = pd.read_csv('title.ratings.tsv', sep='\t')

genre=pd.read_csv('title.crew.tsv', sep='\t')

title_basic=pd.read_csv('title.basics.tsv', sep='\t')


principal=pd.read_csv('title.principals.tsv', sep='\t')

In [ ]:
df_basic = basic.rename(columns={'nconst': 'directors'})

# Facciamo il merge dei due DataFrame
new_df = pd.merge(genre, df_basic, on='directors')

# Rimuoviamo le colonne 'birthYear', 'deathYear', 'writers', 'primaryProfession' dal DataFrame
new_df = new_df.drop(columns=['birthYear', 'deathYear', 'writers', 'primaryProfession'])

new_df = pd.merge(new_df, ratings, on='tconst')

new_df = pd.merge(new_df, title_basic, on='tconst')
new_df = new_df.drop(columns=['titleType', 'originalTitle', 'isAdult','knownForTitles','endYear','runtimeMinutes'])


# Creiamo una lista con le categorie di interesse
categories_of_interest = ['actor', 'actress']

# Filtriamo il DataFrame
principal = principal[principal['category'].isin(categories_of_interest)]
principal = principal.rename(columns={'nconst': 'actors'})
new_df = pd.merge(new_df, principal, on='tconst')
new_df=new_df.drop(columns=['ordering','job','characters','category'])


other_columns = [col for col in new_df.columns if col != 'actors']

# Raggruppa per tutte le altre colonne e unisci gli 'actors' con una virgola
df= new_df.groupby(other_columns)['actors'].agg(', '.join).reset_index()

# Creare un dizionario che mappa i codici unici agli attori
actor_dict = dict(zip(basic['nconst'], basic['primaryName']))

# Creare una nuova colonna 'actor_names' in df mappando i codici agli attori
df['actor_names'] = df['actors'].apply(lambda x: ','.join(actor_dict.get(i, '') for i in x.split(', ')))

df.to_csv('preprocessed_data.csv', index=False)